## Ejercicio 3

### método asignado en la planilla de grupos: MONTECARLO

Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos. Se estima que el tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media =4 segundos

Se debe decidir la arquitectura de base de datos a utilizar entre las dos siguientes:
1. Utilizar 2 bases de datos distribuidas.
Con probabilidad p=0.6 las solicitudes son atendidas por la base 1 y con probabilidad q=1-p son atendidos por la base de datos 2.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias, 1=0,7 seg y 2=1 seg respectivamente.
2. Utilizar 1 base de datos central.
En este caso la demora en resolver una solicitud sigue una distribución exponencial con =0,8 segundos. 

Se suponen despreciables cualquier otro tiempo en el sistema.
Simular para cada opción 100000 solicitudes procesadas, determinando:
- El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión cae
por timeout).
- La fracción de las solicitudes que no esperaron para ser procesadas.
- La opción 1 es más costosa que la segunda opción y la empresa sólo acepta realizar la inversión si el tiempo medio que demora en resolver cada solicitud (tiempo en fila + tiempo de procesamiento) es como mínimo 50% menor
que la opción 2. ¿Qué solución le recomienda?

Proponer otras frecuencias de arribos de solicitudes y comprar los resultados obtenidos.

### Imports

In [7]:
from MixMax import MixmaxRNG
import numpy as np
from collections import deque

### Constantes

In [8]:
# tiempo entre llamadas al servicio
MEAN_ARRIVAL_CALL = 4 

# BASE DE DATOS 1
MEAN_PROCESSING_TIME_DB1 = 0.7  # Tiempo medio de procesamiento en la base de datos 1
MEAN_PROCESING_TIME_DB2 = 1.0  # Tiempo medio de procesamiento en la base de datos 2
PROBABILITY_DB1 = 0.6 # Probabilidad de ser atendido por la base de datos 1 (en la opcion donde hay dos bases de datos distribuidas)

# BASE DE DATOS 2
# Demora en resolver una solicitud -> distribución exponencial con =0,8 segundos
MEAN_PROCESSING_TIME_CENTRAL_DB = 0.8

# semillas
# semilla de arrivos de solicitudes
SEED_ARRIVALS = 1235
# semilla de tiempo de procesamiento de la base de datos
SEED_PROCESING_TIME = 782
# semilla para calculos de probabilidad
SEED = 987654321

TOTAL_REQUEST = 100000


### Código

In [9]:
class DistribuitedDataBase:
    def __init__(self, mean_process_time_d1, mean_process_time_d2, seed, probability_d1):
        self.mean_process_time_d1 = mean_process_time_d1
        self.mean_process_time_d2 = mean_process_time_d2
        self.probability_d1 = probability_d1
        self.pending_calls_d1_queue = deque()
        self.pending_calls_d2_queue = deque()
        self.random = MixmaxRNG(seed)
        # Current time va acumulando arrival + process time y como cada db se maneja con colas distintas
        # mantengo tiemos distintos
        self.current_time_d1 = 0
        self.current_time_d2 = 0
        self.arrival_times = [] 
        self.processing_times = [] 
        self.wait_times = []

    def add_request(self, arrival_time, process_time):
        self.arrival_times.append(arrival_time)
        self.processing_times.append(process_time)
       
        if self.random.generate_number() < self.probability_d1:  
            self.pending_calls_d1_queue.append((arrival_time, process_time))
        else:  
             self.pending_calls_d2_queue.append((arrival_time, process_time))    

    def process_requests(self):
        self.process_requests_d1()
        self.process_requests_d2()

    def process_requests_d1(self):
        # Evaluo todas las requests generadas en la base 1
        while self.pending_calls_d1_queue:
            arrival_time, process_time = self.pending_calls_d1_queue.popleft()
            
            if self.current_time_d1 < arrival_time:
                    self.wait_times.append(0)
                    self.current_time_d1 = arrival_time
            else:
                self.wait_times.append(self.current_time_d1 - arrival_time)
            
            self.current_time_d1 += process_time
    
    def process_requests_d2(self):
        # Evaluo todas las requests generadas en la base 2
        while self.pending_calls_d2_queue:
            arrival_time, process_time = self.pending_calls_d2_queue.popleft()
            
            if self.current_time_d2 < arrival_time:
                    self.wait_times.append(0)
                    self.current_time_d2 = arrival_time
            else:
                self.wait_times.append(self.current_time_d2 - arrival_time)
            
            self.current_time_d2 += process_time

    def get_info(self):
        return self.arrival_times, self.processing_times, self.wait_times

In [10]:
class CentralDataBase:
    def __init__(self, mean_process_time):
        self.mean_process_time = mean_process_time
        self.pending_calls_queue = deque()  # Cola FIFO
        self.arrival_times = [] 
        self.processing_times = [] 
        self.wait_times = []
        self.current_time = 0 # Tiene en cuenta los tiempos de llegada + procesamiento

    def add_request(self, arrival_time, process_time):
        self.arrival_times.append(arrival_time)
        self.processing_times.append(process_time)
        self.pending_calls_queue.append((arrival_time, process_time))

    def process_requests(self):
        # Evaluo todas las requests generadas
        while self.pending_calls_queue:
            # Me quedo con la siguiente en fila
            arrival_time, process_time = self.pending_calls_queue.popleft()
            # Si la siguiente a procesar llega en un tiempo mayor al actual, se procesa
            # (current time se calcula como el arrival + procces time, entonces llegar luego del actual
            #  significa que ya dejo de procesar)
            if self.current_time < arrival_time:
                    self.wait_times.append(0)
                    self.current_time = arrival_time
            # Llega antes del tiempo actual, espera
            else:
                self.wait_times.append(self.current_time - arrival_time)
            
            self.current_time += process_time

    def get_info(self):
        return self.arrival_times, self.processing_times, self.wait_times


In [11]:
class WebService:
    def __init__(self, seed, seed_arrivals, seed_process, TOTAL_REQUEST, mean_time_between_calls, mean_process_times_list, centralDB = True, probability_d1 = 0):
        self.total_requests = TOTAL_REQUEST
        self.mean_time_between_calls = mean_time_between_calls
        self.mean_process_time_d1 = mean_process_times_list[0]
        self.random = MixmaxRNG(seed)
        self.cetral_data_base = centralDB
        self.probability_d1 = probability_d1
        self.random_arrival = MixmaxRNG(seed_arrivals)
        self.random_processing = MixmaxRNG(seed_process)

        if self.cetral_data_base:
            self.dataBase = CentralDataBase(self.mean_process_time_d1)
        else:
            self.mean_process_time_d2 = mean_process_times_list[1]
            self.dataBase = DistribuitedDataBase(self.mean_process_time_d1, self.mean_process_time_d2, seed, self.probability_d1)

    def exponential_time(self, random_num, mean):
         return -(mean * np.log(1 - random_num))

    # Generar llamadas, cada base de datos se encarga de procesarlas
    def generate_and_process_requests(self):
        arrival_time = 0

        for _ in range(self.total_requests):
            # Genero todas las requests, van llegando en orden ya se van sumando los tiempos   
            arrival_time += self.exponential_time(self.random_arrival.generate_number(), self.mean_time_between_calls)            
            process_time = self.exponential_time(self.random_processing.generate_number(), self.mean_process_time_d1)
            self.dataBase.add_request(arrival_time, process_time)

        self.dataBase.process_requests()

    # habia pensado dos func distintas para cada opcionde base de datos pero creo q se
    # puede generalizar en esta
    def generate_and_process_requests_ddb(self):
        arrival_time = 0

        for _ in range(self.total_requests):
            # Genero todas las requests, van llegando en orden ya se van sumando los tiempos
            time_since_last_arrival = self.exponential_time(self.random_arrival.generate_number(), self.mean_time_between_calls)
            arrival_time += time_since_last_arrival
            # si estoy con DDB y la proba < 0.6 o si estoy en CDB y solo existe d1
            if self.random.generate_number() < self.probability_d1 or self.cetral_data_base:
                process_time = self.exponential_time(self.random_processing.generate_number(), self.mean_process_time_d1)
            else:
                process_time = self.exponential_time(self.random_processing.generate_number(), self.mean_process_time_d2)
            # print("Time since last arrival: " + str(time_since_last_arrival) + ", time to process: " + str(process_time))
            self.dataBase.add_request(arrival_time, process_time)

        self.dataBase.process_requests()
        
    def get_info_db(self):
        return self.dataBase.get_info()

### Simulación caso 2 bases de datos distribuidas

todavia no mirar esto :)

In [12]:
# DATOS BASE DE DATOS DISTRIBUIDA
mean_process_times = [MEAN_PROCESSING_TIME_DB1, MEAN_PROCESING_TIME_DB2]
webServiceDistribuited = WebService(SEED, SEED_ARRIVALS, SEED_PROCESING_TIME, TOTAL_REQUEST, MEAN_ARRIVAL_CALL, mean_process_times, centralDB=False, probability_d1=PROBABILITY_DB1)

webServiceDistribuited.generate_and_process_requests_ddb()
arrival_times_ddb, processing_times_ddb, wait_times_ddb = webServiceDistribuited.get_info_db()

# tiempo medio de espera
mean_waiting_time_ddb = sum(wait_times_ddb) / TOTAL_REQUEST
# fracción de solicitudes que no esperaron para ser procesadas
num_no_waiting_ddb = sum(1 for wait_time in wait_times_ddb if wait_time == 0)
fraction_no_waiting_ddb = num_no_waiting_ddb / TOTAL_REQUEST
# tiempo medio de resolucion
total_resolving_time_ddb = sum(processing_times_ddb) + sum(wait_times_ddb)
mean_total_resolving_time_ddb =  total_resolving_time_ddb / TOTAL_REQUEST


print("BASE DE DATOS DISTRIBUIDA")
print(f"el tiempo medio de espera es {mean_waiting_time_ddb}")
print(f"la fraccion de los que no esperan es {fraction_no_waiting_ddb}")
print(f"el tiempo medio del total de resolucion es {mean_total_resolving_time_ddb}")

BASE DE DATOS DISTRIBUIDA
el tiempo medio de espera es 0.09069567095461877
la fraccion de los que no esperan es 0.89768
el tiempo medio del total de resolucion es 0.9097587416233561


In [13]:
# DATOS BASE DE DATOS CENTRAL
webServiceCentral = WebService(SEED, SEED_ARRIVALS, SEED_PROCESING_TIME, TOTAL_REQUEST,  MEAN_ARRIVAL_CALL, [MEAN_PROCESSING_TIME_CENTRAL_DB], centralDB=True, probability_d1=0)

webServiceCentral.generate_and_process_requests()
arrival_times_cdb, processing_times_cdb, wait_times_cdb = webServiceCentral.get_info_db()

# tiempo medio de espera
mean_waiting_time_cdb = sum(wait_times_cdb) / TOTAL_REQUEST
# fracción de solicitudes que no esperaron para ser procesadas
num_no_waiting_cdb = sum(1 for wait_time in wait_times_cdb if wait_time == 0)
fraction_no_waiting_cdb = num_no_waiting_cdb / TOTAL_REQUEST
# tiempo medio de resolucion
total_resolving_time_cdb = sum(processing_times_cdb) + sum(wait_times_cdb)
mean_total_resolving_time_cdb =  total_resolving_time_cdb / TOTAL_REQUEST


print("BASE DE DATOS CENTRAL")
print(f"el tiempo medio de espera es {mean_waiting_time_cdb}")
print(f"la fraccion de los que no esperan es {fraction_no_waiting_cdb}")
print(f"el tiempo medio del total de resolucion es {mean_total_resolving_time_cdb}")

BASE DE DATOS CENTRAL
el tiempo medio de espera es 0.1976976143254088
la fraccion de los que no esperan es 0.80017
el tiempo medio del total de resolucion es 0.9954602018094317
